In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('SYRCityline_Requests_(2021-Present)_Cleaned.csv')

In [3]:
sentiment = pd.read_csv('sentimentResults.csv')

In [4]:
#adding the sentiment analysis columns to the dataframe
df['word_scores'] = sentiment['word_scores']
df['compound'] = sentiment['compound'] 
df['pos'] = sentiment['pos']
df['neu'] = sentiment['neu']
df['neg'] = sentiment['neg']

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112858 entries, 0 to 112857
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Unnamed: 0              112858 non-null  int64  
 1   X                       112858 non-null  float64
 2   Y                       112858 non-null  float64
 3   Id                      112858 non-null  float64
 4   Summary                 112858 non-null  object 
 5   Rating                  112858 non-null  int64  
 6   Address                 112858 non-null  object 
 7   Description             97802 non-null   object 
 8   Agency_Name             112858 non-null  object 
 9   Request_type            112858 non-null  int64  
 10  Lat                     112858 non-null  float64
 11  Lng                     112858 non-null  float64
 12  Created_at_local        112858 non-null  object 
 13  Acknowledged_at_local   21844 non-null   object 
 14  Closed_at_local     

In [6]:
df.head()

,Unnamed: 0,X,Y,Id,Summary,Rating,Address,Description,Agency_Name,Request_type,...,Assignee_name,Category,Sla_in_hours,Report_Source,ObjectId,word_scores,compound,pos,neu,neg
0,1,-8.472894e+06,5.319007e+06,18198655.0,Sewer Back-ups (INTERNAL),1,"435 Ellis St Syracuse, New York, 13210",Dina 450-3461,Water & Sewage,35820,...,Assistant Superintendent of Sewers,Sewer Back-ups (INTERNAL),24.0,Request Form,1,"{0, 0, 0, 0, 0}",0.000,0.00,1.000,0.000
1,2,-8.479295e+06,5.320378e+06,18214945.0,Other Sewer-related Concerns,1,"609 N Geddes St Syracuse, New York, 13204",Jet Flush- Jerry 439-6961,Water & Sewage,33840,...,Assistant Superintendent of Sewers,Other Sewer-related Concerns,24.0,Request Form,2,"{-1.9, 0}",-0.440,0.00,0.256,0.744
2,3,-8.476745e+06,5.313388e+06,18214845.0,Report Improperly Set Out Trash or Recycling (...,1,"3138 Midland Ave Syracuse NY 13205, United States",NaN,"Garbage, Recycling & Graffiti",34425,...,Public Works Inspector- SP,Report Improperly Set Out Trash or Recycling (...,72.0,iPhone,3,"{0, 0, 0, -1.3, 0, 0, 0, -1.9, 0, 0, 0, 0, 0, ...",-0.796,0.00,0.616,0.384
3,4,-8.477781e+06,5.324004e+06,18214835.0,Traffic & Parking Signs,1,"406 Willumae Dr Syracuse NY 13208, United States",Damaged sign down,Streets & Transportation,34423,...,Superintendent of Transportation,Traffic & Parking Signs,120.0,iPhone,4,"{0, 2.9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",0.599,0.17,0.830,0.000
4,5,-8.475632e+06,5.320393e+06,18214821.0,Report an illegally parked vehicle,1,"300-300 Green St Syracuse, NY, 13203, USA",Cars parked illegally and not following odd ev...,Parking & Vehicles,33844,...,Syracuse Police Ordinance,Report an illegally parked vehicle,72.0,Portal,5,"{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.1, 0, 0}",-0.477,0.00,0.819,0.181


In [7]:
# Define the text to replace and its replacement
old_text = "To report an illegally parked vehicle, please call the Syracuse Police Ordinance at 315-448-8650. If this is an emergency, please call 911. Do NOT submit requests to Cityline."
new_text = "Illegally Parked Vehicle"

# Replace the values in the Category column
df['Category'] = df['Category'].replace(old_text, new_text)

# Converting SLA to minutes
df['Sla_in_minutes'] = df['Sla_in_hours'] * 60

# Calculate which requests met/missed SLA
df['Met_Close_SLA'] = df['Minutes_to_Close'] <= df['Sla_in_minutes']
df['Met_Acknowledge_SLA'] = df['Minutes_to_Acknowledge'] <= df['Sla_in_minutes']

# Basic statistics of response times vs SLA
response_stats = pd.DataFrame({
    'Metric': ['Closure Time', 'Acknowledgment Time'],
    'Mean (minutes)': [df['Minutes_to_Close'].mean(), df['Minutes_to_Acknowledge'].mean()],
    'Median (minutes)': [df['Minutes_to_Close'].median(), df['Minutes_to_Acknowledge'].median()],
    'SLA Compliance Rate': [
        (df['Met_Close_SLA'].sum() / df['Met_Close_SLA'].count()) * 100,
        (df['Met_Acknowledge_SLA'].sum() / df['Met_Acknowledge_SLA'].count()) * 100
    ]
})

# Analysis by category
category_performance = df.groupby('Category').agg({
    'Met_Close_SLA': ['count', 'mean'],
    'Minutes_to_Close': 'mean',
    'Sla_in_minutes': 'mean'
}).round(2)

In [8]:
df.head()

,Unnamed: 0,X,Y,Id,Summary,Rating,Address,Description,Agency_Name,Request_type,...,Report_Source,ObjectId,word_scores,compound,pos,neu,neg,Sla_in_minutes,Met_Close_SLA,Met_Acknowledge_SLA
0,1,-8.472894e+06,5.319007e+06,18198655.0,Sewer Back-ups (INTERNAL),1,"435 Ellis St Syracuse, New York, 13210",Dina 450-3461,Water & Sewage,35820,...,Request Form,1,"{0, 0, 0, 0, 0}",0.000,0.00,1.000,0.000,1440.0,False,False
1,2,-8.479295e+06,5.320378e+06,18214945.0,Other Sewer-related Concerns,1,"609 N Geddes St Syracuse, New York, 13204",Jet Flush- Jerry 439-6961,Water & Sewage,33840,...,Request Form,2,"{-1.9, 0}",-0.440,0.00,0.256,0.744,1440.0,False,False
2,3,-8.476745e+06,5.313388e+06,18214845.0,Report Improperly Set Out Trash or Recycling (...,1,"3138 Midland Ave Syracuse NY 13205, United States",NaN,"Garbage, Recycling & Graffiti",34425,...,iPhone,3,"{0, 0, 0, -1.3, 0, 0, 0, -1.9, 0, 0, 0, 0, 0, ...",-0.796,0.00,0.616,0.384,4320.0,False,False
3,4,-8.477781e+06,5.324004e+06,18214835.0,Traffic & Parking Signs,1,"406 Willumae Dr Syracuse NY 13208, United States",Damaged sign down,Streets & Transportation,34423,...,iPhone,4,"{0, 2.9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0...",0.599,0.17,0.830,0.000,7200.0,False,False
4,5,-8.475632e+06,5.320393e+06,18214821.0,Report an illegally parked vehicle,1,"300-300 Green St Syracuse, NY, 13203, USA",Cars parked illegally and not following odd ev...,Parking & Vehicles,33844,...,Portal,5,"{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -2.1, 0, 0}",-0.477,0.00,0.819,0.181,4320.0,False,False


In [9]:
import pandas as pd
import numpy as np

def analyze_acknowledgment_times(df):
    """
    Analyze acknowledgment times across different dimensions using median
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing the SYRCityline requests data
    
    Returns:
    dict: Dictionary containing various analysis results
    """
    
    # Overall median acknowledgment time
    overall_median = df['Minutes_to_Acknowledge'].median()
    
    # Analysis by Agency
    agency_analysis = df.groupby('Agency_Name')['Minutes_to_Acknowledge'].agg([
        ('median_time', 'median'),
        ('count', 'count'),
        ('min_time', 'min'),
        ('max_time', 'max')
    ]).round(2)
    
    # Analysis by Assignee
    assignee_analysis = df.groupby('Assignee_name')['Minutes_to_Acknowledge'].agg([
        ('median_time', 'median'),
        ('count', 'count'),
        ('min_time', 'min'),
        ('max_time', 'max')
    ]).round(2)
    
    # Analysis by Category
    category_analysis = df.groupby('Category')['Minutes_to_Acknowledge'].agg([
        ('median_time', 'median'),
        ('count', 'count'),
        ('min_time', 'min'),
        ('max_time', 'max')
    ]).round(2)
    
    # Filter out groups with very few requests and unrealistically quick median times
    min_requests = 50
    min_median_time = 5
    
    agency_analysis_filtered = agency_analysis[
        (agency_analysis['count'] >= min_requests) & 
        (agency_analysis['median_time'] >= min_median_time)
    ]
    
    assignee_analysis_filtered = assignee_analysis[
        (assignee_analysis['count'] >= min_requests) & 
        (assignee_analysis['median_time'] >= min_median_time)
    ]
    
    category_analysis_filtered = category_analysis[
        (category_analysis['count'] >= min_requests) & 
        (category_analysis['median_time'] >= min_median_time)
    ]
    
    # Sort for both fastest and slowest
    agency_fastest = agency_analysis_filtered.sort_values('median_time', ascending=True)
    agency_slowest = agency_analysis_filtered.sort_values('median_time', ascending=False)
    
    assignee_fastest = assignee_analysis_filtered.sort_values('median_time', ascending=True)
    assignee_slowest = assignee_analysis_filtered.sort_values('median_time', ascending=False)
    
    category_fastest = category_analysis_filtered.sort_values('median_time', ascending=True)
    category_slowest = category_analysis_filtered.sort_values('median_time', ascending=False)
    
    
    return {
        'overall_median': overall_median,
        'agency_fastest': agency_fastest,
        'agency_slowest': agency_slowest,
        'assignee_fastest': assignee_fastest,
        'assignee_slowest': assignee_slowest,
        'category_fastest': category_fastest,
        'category_slowest': category_slowest
    }


In [10]:
import os
from datetime import datetime

# Create the directory if it doesn't exist
output_dir = "Acknowledgement Time Analysis"
os.makedirs(output_dir, exist_ok=True)

# Generate timestamp for the file name
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = os.path.join(output_dir, f'acknowledgment_analysis_{timestamp}.txt')

# Run the analysis
results = analyze_acknowledgment_times(df)

# Save results to file
with open(output_file, 'w') as f:
    f.write("\nOverall Median Acknowledgment Time (minutes): {}\n".format(round(results['overall_median'], 2)))
    
    f.write("\n=== AGENCIES ===\n")
    f.write("\nTop 5 Fastest Responding Agencies:\n")
    f.write(results['agency_fastest'].head().to_string())
    f.write("\n\nTop 5 Slowest Responding Agencies:\n")
    f.write(results['agency_slowest'].head().to_string())
    
    f.write("\n\n=== ASSIGNEES ===\n")
    f.write("\nTop 5 Fastest Responding Assignees:\n")
    f.write(results['assignee_fastest'].head().to_string())
    f.write("\n\nTop 5 Slowest Responding Assignees:\n")
    f.write(results['assignee_slowest'].head().to_string())
    
    f.write("\n\n=== CATEGORIES ===\n")
    f.write("\nTop 5 Fastest Acknowledged Categories:\n")
    f.write(results['category_fastest'].head().to_string())
    f.write("\n\nTop 5 Slowest Acknowledged Categories:\n")
    f.write(results['category_slowest'].head().to_string())
    
    f.write("\n\nSummary Statistics for Acknowledgment Times (in minutes):\n")
    f.write(df['Minutes_to_Acknowledge'].describe().round(2).to_string())
    
    # Calculate percentage of requests acknowledged within different time windows
    time_windows = [60, 120, 240, 480, 1440]  # 1 hour, 2 hours, 4 hours, 8 hours, 24 hours
    for window in time_windows:
        pct = (df['Minutes_to_Acknowledge'] <= window).mean() * 100
        f.write(f"\n\nPercentage of requests acknowledged within {window} minutes: {pct:.2f}%")

print(f"Analysis results have been saved to: {output_file}")

Analysis results have been saved to: Acknowledgement Time Analysis\acknowledgment_analysis_20250223_103221.txt


In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
from datetime import datetime

# Set style and color preferences
plt.style.use('bmh')
sns.set_context("notebook", font_scale=1.2)

def plot_acknowledgment_analysis(df, min_requests=10, min_median_time=5):
    """
    Create and save visualizations showing fastest and slowest response times
    """
    # Create directory if it doesn't exist
    output_dir = "Acknowledgement Time Analysis"
    os.makedirs(output_dir, exist_ok=True)
    
    # Generate timestamp for unique filenames
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. Top 5 Slowest Agencies (Red)
    plt.figure(figsize=(12, 6))
    agency_stats = df.groupby('Agency_Name')['Minutes_to_Acknowledge'].agg(['median', 'count'])
    agency_stats = agency_stats[
        (agency_stats['count'] >= min_requests) & 
        (agency_stats['median'] >= min_median_time)
    ]
    
    top_5_agencies = agency_stats.nlargest(5, 'median')
    colors = sns.color_palette("Reds_r", n_colors=5)
    ax = sns.barplot(x=top_5_agencies['median'], y=top_5_agencies.index, 
                    palette=colors, orient='h')
    plt.title('5 Slowest Responding Agencies', pad=20)
    plt.xlabel('Median Minutes to Acknowledge')
    for i, v in enumerate(top_5_agencies['median']):
        ax.text(v, i, f'{v:,.0f}', va='center')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'slowest_agencies_{timestamp}.png'), 
                bbox_inches='tight', dpi=300)
    plt.close()

    # 2. Top 5 Fastest Agencies (Green)
    plt.figure(figsize=(12, 6))
    bottom_5_agencies = agency_stats.nsmallest(5, 'median')
    colors = sns.color_palette("Greens_r", n_colors=5)  # Changed to Greens_r
    ax = sns.barplot(x=bottom_5_agencies['median'], y=bottom_5_agencies.index[::-1], 
                    palette=colors, orient='h')
    plt.title('5 Fastest Responding Agencies', pad=20)
    plt.xlabel('Median Minutes to Acknowledge')
    for i, v in enumerate(bottom_5_agencies['median']):
        ax.text(v, i, f'{v:,.0f}', va='center')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'fastest_agencies_{timestamp}.png'), 
                bbox_inches='tight', dpi=300)
    plt.close()

    # 3. Top 10 Slowest Assignees (Red)
    plt.figure(figsize=(12, 8))
    assignee_stats = df.groupby('Assignee_name')['Minutes_to_Acknowledge'].agg(['median', 'count'])
    assignee_stats = assignee_stats[
        (assignee_stats['count'] >= min_requests) & 
        (assignee_stats['median'] >= min_median_time)
    ]
    
    top_10_assignees = assignee_stats.nlargest(10, 'median')
    colors = sns.color_palette("Reds_r", n_colors=10)
    ax = sns.barplot(x=top_10_assignees['median'], y=top_10_assignees.index, 
                    palette=colors, orient='h')
    plt.title('10 Slowest Responding Assignees', pad=20)
    plt.xlabel('Median Minutes to Acknowledge')
    for i, v in enumerate(top_10_assignees['median']):
        ax.text(v, i, f'{v:,.0f}', va='center')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'slowest_assignees_{timestamp}.png'), 
                bbox_inches='tight', dpi=300)
    plt.close()

    # 4. Top 10 Fastest Assignees (Green)
    plt.figure(figsize=(12, 8))
    bottom_10_assignees = assignee_stats.nsmallest(10, 'median')
    colors = sns.color_palette("Greens_r", n_colors=10)  # Changed to Greens_r
    ax = sns.barplot(x=bottom_10_assignees['median'], y=bottom_10_assignees.index[::-1], 
                    palette=colors, orient='h')
    plt.title('10 Fastest Responding Assignees', pad=20)
    plt.xlabel('Median Minutes to Acknowledge')
    for i, v in enumerate(bottom_10_assignees['median']):
        ax.text(v, i, f'{v:,.0f}', va='center')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'fastest_assignees_{timestamp}.png'), 
                bbox_inches='tight', dpi=300)
    plt.close()

    # 5. Top 10 Slowest Categories (Red)
    plt.figure(figsize=(12, 8))
    category_stats = df.groupby('Category')['Minutes_to_Acknowledge'].agg(['median', 'count'])
    category_stats = category_stats[
        (category_stats['count'] >= min_requests) & 
        (category_stats['median'] >= min_median_time)
    ]
    
    top_10_categories = category_stats.nlargest(10, 'median')
    colors = sns.color_palette("Reds_r", n_colors=10)
    ax = sns.barplot(x=top_10_categories['median'], y=top_10_categories.index, 
                    palette=colors, orient='h')
    plt.title('10 Slowest Response Categories', pad=20)
    plt.xlabel('Median Minutes to Acknowledge')
    for i, v in enumerate(top_10_categories['median']):
        ax.text(v, i, f'{v:,.0f}', va='center')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'slowest_categories_{timestamp}.png'), 
                bbox_inches='tight', dpi=300)
    plt.close()

    # 6. Top 10 Fastest Categories (Green)
    plt.figure(figsize=(12, 8))
    bottom_10_categories = category_stats.nsmallest(10, 'median')
    colors = sns.color_palette("Greens_r", n_colors=10)  # Changed to Greens_r
    ax = sns.barplot(x=bottom_10_categories['median'], y=bottom_10_categories.index[::-1], 
                    palette=colors, orient='h')
    plt.title('10 Fastest Response Categories', pad=20)
    plt.xlabel('Median Minutes to Acknowledge')
    for i, v in enumerate(bottom_10_categories['median']):
        ax.text(v, i, f'{v:,.0f}', va='center')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'fastest_categories_{timestamp}.png'), 
                bbox_inches='tight', dpi=300)
    plt.close()
    
    print(f"All plots have been saved to the '{output_dir}' folder with timestamp {timestamp}")

# Run the visualization function
plot_acknowledgment_analysis(df)

C:\Users\itsni\AppData\Local\Temp\ipykernel_21120\646633183.py:33: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.barplot(x=top_5_agencies['median'], y=top_5_agencies.index,
C:\Users\itsni\AppData\Local\Temp\ipykernel_21120\646633183.py:48: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.barplot(x=bottom_5_agencies['median'], y=bottom_5_agencies.index[::-1],
C:\Users\itsni\AppData\Local\Temp\ipykernel_21120\646633183.py:69: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  ax = sns.barplot(x=top_10_assignees['median'], y=top_10_assignees.index,
C:\Users\itsni\AppDat

All plots have been saved to the 'Acknowledgement Time Analysis' folder with timestamp 20250223_103221


In [12]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import gaussian_filter
import pandas as pd

# First ensure the datetime column is properly converted
df['Created_at_local'] = pd.to_datetime(df['Created_at_local'])

# Create the heatmap data
df['hour'] = df['Created_at_local'].dt.hour
df['day_of_week'] = df['Created_at_local'].dt.day_name()

# Create pivot table
heatmap_data = df.pivot_table(
    values='Minutes_to_Acknowledge',
    index='day_of_week',
    columns='hour',
    aggfunc='median'
)

# Reorder days
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
heatmap_data = heatmap_data.reindex(day_order)

# Filter for hours from 8am onwards
heatmap_data = heatmap_data.loc[:, 8:23]

# Convert to numpy array
data = heatmap_data.to_numpy()

# Apply Gaussian smoothing
smoothed_data = gaussian_filter(data, sigma=1)

# Create the plot
plt.figure(figsize=(12, 8))

# Create smooth heatmap with correct orientation
plt.imshow(smoothed_data, aspect='auto', cmap='YlOrRd', 
           extent=[8, 23, -0.5, 6.5], origin='lower')

colorbar = plt.colorbar(label='Median Minutes to Acknowledge')

# Customize axes
plt.yticks(range(7), ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
plt.xticks([8, 11, 14, 17, 20, 23], ['8am', '11am', '2pm', '5pm', '8pm', '11pm'])
plt.title('Response Time Patterns Throughout Week', pad=20)
plt.xlabel('Time of Day')
plt.ylabel('Day of Week')

# Remove grid
plt.grid(False)
plt.tight_layout()

# Save the plot
output_dir = "Acknowledgement Time Analysis"
os.makedirs(output_dir, exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
plt.savefig(os.path.join(output_dir, f'response_time_heatmap_{timestamp}.png'), 
            bbox_inches='tight', dpi=300)
plt.close()